# 逻辑设计

## 目标和前提条件

在本例中，你将学习如何解决一个逻辑设计问题，该问题涉及使用最少数量的NOR门（具有两个输入和一个输出的设备）构建一个电路，以执行由真值表指定的逻辑功能。我们将向你展示如何使用Gurobi Python API将此问题表述为二进制优化问题，然后使用Gurobi优化器自动找到最优解。

此模型是H. Paul Williams所著《Model Building in Mathematical Programming》第五版第266-267页和320-321页的示例12。

这是一个中级示例，我们假设你了解Python和Gurobi Python API，并且对构建数学优化模型有一定了解。

**下载代码库** <br />
你可以通过点击[这里](https://github.com/Gurobi/modeling-examples/archive/master.zip)下载包含此示例和其他示例的代码库。

## 问题描述

逻辑电路具有给定数量的输入和一个输出。向给定逻辑电路的输入施加脉冲时，它将通过给出输出（信号1）或无输出（信号0）来响应。输入脉冲与输出类型相同 - 1（正输入）或0（无输入）。

在本例中，将使用NOR门构建逻辑电路。NOR门是一个具有两个输入和一个输出的设备。它具有这样的特性：当且仅当两个输入都为0时（即两个输入都没有正输入），才会有正输出（信号1）。通过将这些门连接在一起，一个门的输出可能是另一个门的输入，可以构建一个执行任何所需逻辑功能的电路。例如，下图所示的电路将按真值表所示的方式响应输入A和B。
![circuit](circuit.PNG)

目标是使用最少数量的NOR门构建一个电路，以执行由以下真值表指定的逻辑功能。

| A | B | 输出 |
| --- | --- | --- |
| 0 | 0 | 0 |
| 0 | 1 | 1 |
| 1 | 0 | 1 |
| 1 | 1 | 0 |

不允许'扇入'和'扇出'。也就是说，一个NOR门的多个输出不能引入一个输入，一个输出也不能引入多个输入。可以假设最优设计是下图所示'最大'网络的'子网'。
![subnet](subnet.PNG)

## 模型构建

### 集合和索引

$i \in \text{Gates}=\{1,...,7\}$

$i \in \text{G47}=\{4,...,7\}$

$r \in \text{Rows}=\{1,...,4\}$

### 参数

$\text{valueA}_{i,r} \in \{0,1 \}$：真值表第r行中门i的外部输入A的值。

$\text{valueB}_{i,r} \in \{0,1 \}$：真值表第r行中门i的外部输入B的值。

### 决策变量

$\text{NOR}_{i} \in \{0,1 \}$：如果选择NOR门i，则此二进制变量等于1，否则为0。

$\text{inputA}_{i} \in \{0,1 \}$：如果外部输入A是NOR门i的输入，则此二进制变量等于1，否则为0。

$\text{inputB}_{i} \in \{0,1 \}$：如果外部输入B是NOR门i的输入，则此二进制变量等于1，否则为0。

$\text{output}_{i,r} \in \{0,1 \}$：此二进制变量是门i对于真值表第r行指定的外部输入信号组合的输出。

### 约束条件

**外部输入**：只有当NOR门存在时，它才能有外部输入。

\begin{equation}
\text{NOR}_{i} \geq \text{inputA}_{i}  \quad \forall i \in \text{Gates}
\end{equation}

\begin{equation}
\text{NOR}_{i} \geq \text{inputB}_{i}  \quad \forall i \in \text{Gates}
\end{equation}

**NOR门**：如果一个NOR门有一个（或两个）外部输入，则只能有一个（或零个）NOR门馈入它。

\begin{equation}
\text{NOR}_{2} + \text{NOR}_{3} + \text{inputA}_{1} + \text{inputB}_{1} \leq 2
\end{equation}

\begin{equation}
\text{NOR}_{4} + \text{NOR}_{5} + \text{inputA}_{2} + \text{inputB}_{2} \leq 2
\end{equation}

\begin{equation}
\text{NOR}_{6} + \text{NOR}_{7} + \text{inputA}_{3} + \text{inputB}_{3} \leq 2
\end{equation}

这些约束基于'最大'网络图中所示的电路。

**输出信号**：如果门i存在，则门i的输出信号必须是门i输入信号的正确逻辑函数（NOR）。

\begin{equation}
\text{output}_{2,r} + \text{output}_{1,r} \leq 1 \quad \forall r \in \text{Rows}
\end{equation}

\begin{equation}
\text{output}_{3,r} + \text{output}_{1,r} \leq 1 \quad \forall r \in \text{Rows}
\end{equation}

\begin{equation}
\text{valueA}_{i,r}*\text{inputA}_{i} + \text{output}_{i,r} \leq 1 \quad \forall i \in \text{Gates}, r \in \text{Rows}
\end{equation}

\begin{equation}
\text{valueB}_{i,r}*\text{inputB}_{i} + \text{output}_{i,r} \leq 1 \quad \forall i \in \text{Gates}, r \in \text{Rows}
\end{equation}

\begin{equation}
\text{valueA}_{i,r}*\text{inputA}_{i} + \text{valueB}_{i,r}*\text{inputB}_{i} + 
\text{output}_{i,r} - \text{NOR}_{i}  \geq 0 \quad \forall i \in \text{G47}, r \in \text{Rows}
\end{equation}

\begin{equation}
\text{valueA}_{1,r}*\text{inputA}_{1} + \text{valueB}_{1,r}*\text{inputB}_{1} + 
\text{output}_{2,r} + \text{output}_{3,r} + \text{output}_{1,r} - \text{NOR}_{1}  \geq 0 
\quad \forall r \in \text{Rows}
\end{equation}

\begin{equation}
\text{valueA}_{2,r}*\text{inputA}_{2} + \text{valueB}_{2,r}*\text{inputB}_{2} + 
\text{output}_{4,r} + \text{output}_{5,r} + \text{output}_{2,r} - \text{NOR}_{2}  \geq 0 
\quad \forall r \in \text{Rows}
\end{equation}

\begin{equation}
\text{valueA}_{3,r}*\text{inputA}_{3} + \text{valueB}_{3,r}*\text{inputB}_{3} + 
\text{output}_{6,r} + \text{output}_{7,r} + \text{output}_{3,r} - \text{NOR}_{3}  \geq 0 
\quad \forall r \in \text{Rows}
\end{equation}

**门1**：对于NOR门1，输出变量固定在真值表中指定的值。

\begin{equation}
\text{output}_{1,1} = 0, \text{output}_{1,2} = 1, \text{output}_{1,3} = 1,  \text{output}_{1,4} = 0   
\end{equation}

为了避免不包含NOR门的平凡解，必须施加选择NOR门1的约束。

\begin{equation}
\text{NOR}_{1} \geq 1  
\end{equation}

**门和输出**：如果对于任何输入信号组合，特定NOR门有输出信号1，则该门必须存在。

\begin{equation}
\text{NOR}_{i} - \text{output}_{i,r} \geq 0 \quad \forall i \in \text{Gates}, r \in \text{Rows}
\end{equation}

### 目标函数

**门的数量**：目标是最小化选择的NOR门数量。

\begin{equation}
\text{Minimize} \quad  \sum_{i \in \text{Gates}} \text{NOR}_{i}
\end{equation}

## Python实现

我们导入Gurobi Python模块。

In [ ]:
# %pip install gurobipy

In [ ]:
import gurobipy as gp
from gurobipy import GRB

# 测试环境：Python 3.11 & Gurobi 11.0

## 输入数据

我们定义模型的所有输入数据。

In [ ]:
# NOR门1到7的列表

gates = ['1','2','3','4','5','6','7']

# NOR门4到7的列表

gates47 = ['4','5','6','7']

# 真值表中1到4范围内的行列表

rows = ['1','2','3','4']

# 创建一个字典来记录每个NOR门i在真值表r行中外部输入A和B的值

gatesRows, valueA, valueB = gp.multidict({
    ('1','1'): [0,0],
    ('1','2'): [0,1],
    ('1','3'): [1,0],
    ('1','4'): [1,1],
    ('2','1'): [0,0],
    ('2','2'): [0,1],
    ('2','3'): [1,0],
    ('2','4'): [1,1],
    ('3','1'): [0,0],
    ('3','2'): [0,1],
    ('3','3'): [1,0],
    ('3','4'): [1,1],
    ('4','1'): [0,0],
    ('4','2'): [0,1],
    ('4','3'): [1,0],
    ('4','4'): [1,1],
    ('5','1'): [0,0],
    ('5','2'): [0,1],
    ('5','3'): [1,0],
    ('5','4'): [1,1],
    ('6','1'): [0,0],
    ('6','2'): [0,1],
    ('6','3'): [1,0],
    ('6','4'): [1,1],
    ('7','1'): [0,0],
    ('7','2'): [0,1],
    ('7','3'): [1,0],
    ('7','4'): [1,1]
})


## 模型部署

我们创建一个模型和变量。主要决策是确定选择要在逻辑电路中考虑的NOR门的$\text{NOR}_{i}$变量。其余变量确保电路生成真值表的输出。

In [ ]:
model = gp.Model('logicalDesign')

# 用于选择NOR门i的决策变量
NOR = model.addVars(gates, vtype=GRB.BINARY, name="NORgate" )

# 为了避免一个不包含NOR门的平凡解，需要添加一个约束来选择NOR门1

NOR['1'].lb = 1

# 用于决定外部输入A是否为NOR门i的输入的变量
inputA = model.addVars(gates, vtype=GRB.BINARY, name="inputA")

# 用于决定外部输入B是否为NOR门i的输入的变量
inputB = model.addVars(gates, vtype=GRB.BINARY, name="inputB")

# 输出决策变量
output = model.addVars(gatesRows, vtype=GRB.BINARY, name="output")

# 对于NOR门1，输出变量固定在真值表中指定的值

output['1','1'].ub = 0
output['1','2'].lb = 1
output['1','3'].lb = 1
output['1','4'].ub = 0

Using license file c:\gurobi\gurobi.lic


NOR门只有在存在时才能有外部输入。

In [ ]:
# 外部输入约束

externalInputsA = model.addConstrs( ( NOR[i] >= inputA[i]  for i in gates), name='externalInputsA')

externalInputsB = model.addConstrs( ( NOR[i] >= inputB[i]  for i in gates), name='externalInputsB')

如果一个NOR门有一个（或两个）外部输入导入，则只能有一个（或零个）NOR门馈入它。

In [ ]:
# NOR门约束

NORgate1 = model.addConstr(NOR['2'] + NOR['3'] + inputA['1'] + inputB['1'] <= 2, name='NORgate1')

NORgate2 = model.addConstr(NOR['4'] + NOR['5'] + inputA['2'] + inputB['2'] <= 2, name='NORgate2')

NORgate3 = model.addConstr(NOR['6'] + NOR['7'] + inputA['3'] + inputB['3'] <= 2, name='NORgate3')

如果门i存在，则门i的输出信号必须是输入信号的正确逻辑函数（NOR）。

In [ ]:
# 输出信号约束

outputSignals1_1 = model.addConstrs( (output['2',r] + output['1',r] <= 1 for r in rows), name='outputSignals1_1' )

outputSignals1_2 = model.addConstrs( (output['3',r] + output['1',r] <= 1 for r in rows), name='outputSignals1_2' )

outputSignals2_1 = model.addConstrs( (output['4',r] + output['2',r] <= 1 for r in rows), name='outputSignals2_1' )

outputSignals2_2 = model.addConstrs( (output['5',r] + output['2',r] <= 1 for r in rows), name='outputSignals2_2' )

outputSignals3_1 = model.addConstrs( (output['6',r] + output['3',r] <= 1 for r in rows), name='outputSignals3_1' )

outputSignals3_2 = model.addConstrs( (output['7',r] + output['3',r] <= 1 for r in rows), name='outputSignals3_2' )


outputSignals4 = model.addConstrs( (valueA[i,r]*inputA[i] + output[i,r] <= 1 for i,r in gatesRows), name='outputSignals4')

outputSignals5 = model.addConstrs( (valueB[i,r]*inputB[i] + output[i,r] <= 1 for i,r in gatesRows), name='outputSignals5')

outputSignals6 = model.addConstrs( (valueA[i,r]*inputA[i] + valueB[i,r]*inputB[i] + output[i,r] - NOR[i] >= 0 
                                    for i,r in gatesRows if i in gates47), name='outputSignals6')

outputSignals7 = model.addConstrs( (valueA['1',r]*inputA['1'] + valueB['1',r]*inputB['1'] 
                                    + output['2',r] + output['3',r] + output['1',r] - NOR['1'] >= 0
                                    for i,r in gatesRows), name='outputSignals7')

outputSignals8 = model.addConstrs( (valueA['2',r]*inputA['2'] + valueB['2',r]*inputB['2'] 
                                    + output['4',r] + output['5',r] + output['2',r] - NOR['2'] >= 0
                                    for i,r in gatesRows), name='outputSignals8')

outputSignals9 = model.addConstrs( (valueA['3',r]*inputA['3'] + valueB['3',r]*inputB['3'] 
                                    + output['6',r] + output['7',r] + output['3',r] - NOR['3'] >= 0
                                    for i,r in gatesRows), name='outputSignals9')

如果对于任何输入信号组合，特定NOR门有输出信号1，则该门必须存在。

In [ ]:
# 门和输出信号约束

gateOutput = model.addConstrs( (NOR[i] - output[i,r] >= 0 for i,r in gatesRows) , name='gateOutput')

目标是最小化选择的NOR门数量。

In [ ]:
# 目标函数

model.setObjective(NOR.sum(), GRB.MINIMIZE)

In [ ]:
# 验证模型公式

# model.write('logicalDesign.lp')

# 运行优化引擎

model.optimize()

Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 225 rows, 49 columns and 696 nonzeros
Model fingerprint: 0x9adba516
Variable types: 0 continuous, 49 integer (49 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+00]
Presolve removed 225 rows and 49 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds
Thread count was 1 (of 8 available processors)

Solution count 1: 5 

Optimal solution found (tolerance 1.00e-04)
Best objective 5.000000000000e+00, best bound 5.000000000000e+00, gap 0.0000%


In [ ]:
# 输出报告

print("\n\n_________________________________________________________________________________")
print(f"最优电路设计:")
print("_________________________________________________________________________________")
for i in gates:
    if (NOR[i].x > 0.5):
        if (inputA[i].x + inputB[i].x > 0.5):
            print(f"NOR门 {i} 已激活，外部输入A和B的值分别为 {inputA[i].x} 和 {inputB[i].x}。")
        else:
            print(f"NOR门 {i} 已激活。")




_________________________________________________________________________________
The optimal circuit design:
_________________________________________________________________________________
NOR gate 1 is active.
NOR gate 2 is active, with external inputs A and B values of 1.0 and  1.0.
NOR gate 3 is active.
NOR gate 6 is active, with external inputs A and B values of 0.0 and  1.0.
NOR gate 7 is active, with external inputs A and B values of 1.0 and  0.0.


## 参考文献

H. Paul Williams, 《Model Building in Mathematical Programming》第五版。

Copyright © 2020 Gurobi Optimization, LLC